## Experiments: Comparing RSAST with other methods

In [1]:
# Import the os module
import os
import sys
path = r'C:\Users\Surface pro\random_sast'
try:
    os.chdir(path)
    print("Current working directory: {0}".format(os.getcwd()))
except FileNotFoundError:
    print("Directory: {0} does not exist".format(path))
except NotADirectoryError:
    print("{0} is not a directory".format(path))
except PermissionError:
    print("You do not have permissions to change to {0}".format(path))

sys.path.append(r'C:\Users\Surface pro\random_sast\sast')

Current working directory: C:\Users\Surface pro\random_sast


In [2]:
from sast import *
from sktime.datasets import load_UCR_UEA_dataset, tsc_dataset_names
from sktime.classification.kernel_based import RocketClassifier
import time
import pandas as pd
from sktime.transformations.panel.shapelet_transform import ShapeletTransform
import matplotlib.pyplot as plt
import seaborn as sns
import math
from utils_sast import delete_files_in_directory
from cd_diagram.cd_function import *

c:\Users\Surface pro\random_sast\.venv\lib\site-packages\mass_ts\_mass_ts.py:17: UserWarning: GPU support will not work. You must pip install mass-ts[gpu].
  warnings.warn(


C:\Users\Surface pro\random_sast


c:\Users\Surface pro\random_sast\.venv\lib\site-packages\aeon\transformations\collection\tsfresh.py:13: UserWarning: No module named 'tsfresh'. 'tsfresh' is a soft dependency and not included in the base aeon installation. Please run: `pip install tsfresh` to install the tsfresh package. To install all soft dependencies, run: `pip install aeon[all_extras]`
  _check_soft_dependencies("tsfresh", severity="warning")


### Reading Datasets

In [3]:

# Set directory where the csv files are located
directory = os.getcwd()+'/ExperimentationRSAST/results_other_methods/results_by_classifier'

# Create an empty list to store the dataframes
dfs = []

header=['dataset']
res = [str(ele) for ele in range(30)]
header.extend(res)

# Loop through all files in the directory
for filename in os.listdir(directory):
    # Check if the file starts with "df_overall_tunning" and ends with ".csv"
    if filename.endswith(".csv"):
        # Read the csv file into a dataframe
        filepath = os.path.join(directory, filename)
        df = pd.read_csv(filepath, names=header,index_col=False)

        # Append the dataframe to the list
        df['filename']=filename
        dfs.append(df)


# Concatenate all the dataframes in the list into one dataframe
df_other_methods = pd.concat(dfs, ignore_index=True)

#10 Resampling
res=res[:10]

df_other_methods["avg"]=df_other_methods[res].mean(axis=1)

df_other_methods=df_other_methods[['filename','dataset','avg']]
df_other_methods['method']=df_other_methods['filename'].str.split('.').str[0]+"_UCR_10RES"
df_other_methods
df_other_methods=df_other_methods.rename(columns={'avg':'score'})
df_other_methods=df_other_methods[['dataset','score','method']]


C:\Users\Surface pro\AppData\Local\Temp\ipykernel_16740\4288461203.py:17: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df = pd.read_csv(filepath, names=header,index_col=False)
C:\Users\Surface pro\AppData\Local\Temp\ipykernel_16740\4288461203.py:17: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df = pd.read_csv(filepath, names=header,index_col=False)
C:\Users\Surface pro\AppData\Local\Temp\ipykernel_16740\4288461203.py:17: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df = pd.read_csv(filepath, names=header,index_col=False)
C:\Users\Surface pro\AppData\Local\Temp\ipykernel_16740\4288461203.py:17: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df = pd.read_csv(filepath, names=

In [4]:
# Set directory where the csv files are located

filepath = os.getcwd()+'/ExperimentationRSAST/results_other_methods/results_summarized/UCR_ARCHIVE_singleTrainTest_ALL_METHODS_MOD.csv'

df = pd.read_csv(filepath)
df = df.melt(id_vars=["Unnamed: 0"], 
        var_name="method", 
        value_name="score")
df = df.rename(columns={'Unnamed: 0':'dataset'})
df['method'] = df['method']+"_UCR_TRAIN_TEST"
df=df[['dataset','score','method']]
df_other_methods=pd.concat([df_other_methods,df])


In [5]:

# Set directory where the csv files are located
#df_other_methods=df_other_methods[df_other_methods.method!="STC"]
filepath = os.getcwd()+'/ExperimentationRSAST/results_other_methods/results_summarized/SAST_PAPER_singleTrainTest_SAST_ST.csv'
df = pd.read_csv(filepath)
df = df.rename(columns={'folds:':'dataset','STC':'score'})
df['method']="STC_PAPERSAST_TRAIN_TEST"
df=df[['dataset','score','method']]
df_other_methods=pd.concat([df_other_methods,df])



In [6]:
# Set directory where the csv files are located

filepath = os.getcwd()+'/ExperimentationRSAST/results_other_methods/results_summarized/SAST_PAPER_singleTrainTest_SAST_ST.csv'
df = pd.read_csv(filepath)
df = df.rename(columns={'folds:':'dataset','0':'score'})
df['method']="SAST_PAPERSAST_TRAIN_TEST"
df=df[['dataset','score','method']]
df_other_methods=pd.concat([df_other_methods,df])

In [7]:
# Set directory where the csv files are located

filepath = os.getcwd()+'/ExperimentationRSAST/results_other_methods/results_summarized/ROCKET_PAPER_10RESAMPLES_BAKE_OFF.csv'
df = pd.read_csv(filepath)
df = df.melt(id_vars=["Dataset"], 
        var_name="method", 
        value_name="score")
df = df.rename(columns={'Dataset':'dataset'})
df['method'] = df['method']+"_PAPERROCKET_AVG_10RES"
df=df[['dataset','score','method']]
df_other_methods=pd.concat([df_other_methods,df])


In [8]:
# Set directory where the csv files are located


filepath = os.getcwd()+'/ExperimentationRSAST/results_other_methods/results_summarized/ROCKET_PAPER_10RESAMPLES_2018_DS.csv'
df = pd.read_csv(filepath)
df = df.melt(id_vars=["Dataset"], 
        var_name="method", 
        value_name="score")
df = df.rename(columns={'Dataset':'dataset'})
df['method'] = df['method']+"_PAPERROCKET_AVG_10RES"
df=df[['dataset','score','method']]
df_other_methods=pd.concat([df_other_methods,df])


In [9]:
# Set directory where the csv files are located


filepath = os.getcwd()+'/ExperimentationRSAST/results_other_methods/results_summarized/RDST_PAPER_30RESAMPLES_ALL_METHODS.csv'
df = pd.read_csv(filepath)
df = df.melt(id_vars=["dataset_name"], 
        var_name="method", 
        value_name="score")
df = df.rename(columns={'dataset_name':'dataset'})
df['method'] = df['method']+"_PAPERRDST_AVG_30RES"
df=df[['dataset','score','method']]
df_other_methods=pd.concat([df_other_methods,df])


In [10]:
# Set directory where the csv files are located

filepath = os.getcwd()+'/ExperimentationRSAST/results_other_methods/results_summarized/RDST_PAPER_30RESAMPLES_RDST.csv'
df = pd.read_csv(filepath)
df = df.rename(columns={'dataset':'dataset','acc_mean':'score','model':'method'})
df['method'] = df['method']+"_PAPERRDST_AVG_30RES"
df=df[['dataset','score','method']]
df_other_methods=pd.concat([df_other_methods,df])


In [11]:

# Set directory where the csv files are located
directory = os.getcwd()+'/ExperimentationRSAST/results_other_methods/results_summarized/Server17_Comparison_Other_Methods_singleTrainTest'

# Create an empty list to store the dataframes
dfs = []

# Loop through all files in the directory
for filename in os.listdir(directory):
    # Check if the file starts with "df_overall_tunning" and ends with ".csv"
    if filename.endswith(".csv"):
        # Read the csv file into a dataframe
        filepath = os.path.join(directory, filename)
        df = pd.read_csv(filepath)

        # Append the dataframe to the list
        
        dfs.append(df)

# Concatenate all the dataframes in the list into one dataframe
df = pd.concat(dfs, ignore_index=True)
df=df[['dataset_name','classifier_name','accuracy']]

df=pd.pivot_table(df, index=['dataset_name','classifier_name'],aggfunc='mean').reset_index()
df['method']=df['classifier_name'].str.split('_').str[0]+"_RSAST_TRAIN_TEST"
df=df.rename(columns={'dataset_name':'dataset','accuracy':'score'})

df=df[['dataset','score','method']]
df_other_methods=pd.concat([df_other_methods,df])


In [12]:
# Set directory where the csv files are located
directory = os.getcwd()+'/ExperimentationRSAST/results_other_methods/results_summarized/Server17_Comparison_Other_Methods_Resampling'

# Create an empty list to store the dataframes
dfs = []

# Loop through all files in the directory
for filename in os.listdir(directory):
    # Check if the file starts with "df_overall_tunning" and ends with ".csv"
    if filename.endswith(".csv"):
        # Read the csv file into a dataframe
        filepath = os.path.join(directory, filename)
        df = pd.read_csv(filepath)

        # Append the dataframe to the list
        
        dfs.append(df)

# Concatenate all the dataframes in the list into one dataframe
df = pd.concat(dfs, ignore_index=True)
df=df[['dataset_name','classifier_name','accuracy']]

df=pd.pivot_table(df, index=['dataset_name','classifier_name'],aggfunc='mean').reset_index()
df['method']=df['classifier_name'].str.split('_').str[0]+"_RSAST_10RES"
df=df.rename(columns={'dataset_name':'dataset','accuracy':'score'})

df=df[['dataset','score','method']]
df_other_methods=pd.concat([df_other_methods,df])


In [13]:
# Set directory where the RSAST result csv files are located

directories=[]
default_split_acfandpacf_k_1_10_sast=True
default_split_acfandpacf_k_1_10_notsast=False
default_split_all_k_1_10=False
default_split_all_acfandpacf_k_30_50=False
default_split_maxpacf_none_k_30_50=False
default_split_all_acfandpacf=False
default_split_all_acfandpacf_maxpacf_none_maxacf_k_1_50_p_1_100=False

resampling_acfandpacf_k_1_10_sast=False

if default_split_acfandpacf_k_1_10_sast:
    directories.append(os.getcwd()+'/ExperimentationRSAST/results_rsast/Server17_Comparison_RSAST')
if default_split_acfandpacf_k_1_10_notsast:
    directories.append(os.getcwd()+'/ExperimentationRSAST/results_rsast/Server17_not_used_sast')
if default_split_all_k_1_10:
    directories.append(os.getcwd()+'/ExperimentationRSAST/results_rsast/Server17_Comparison_RSAST_All')   
if default_split_all_acfandpacf_k_30_50:
    directories.append(os.getcwd()+'/ExperimentationRSAST/results_rsast/Server17_Comparison_RSAST_Extra_1')    
if default_split_maxpacf_none_k_30_50:
    directories.append(os.getcwd()+'/ExperimentationRSAST/results_rsast/Server17_Comparison_RSAST_Extra_2')    
if resampling_acfandpacf_k_1_10_sast:
    directories.append(os.getcwd()+'/ExperimentationRSAST/results_rsast/Server17_Resampling_Comparison_RSAST')     
if default_split_all_acfandpacf_maxpacf_none_maxacf_k_1_50_p_1_100:
    directories.append(os.getcwd()+'/ExperimentationRSAST/results_rsast/Server16_Hyperparameter_Tunning')
    directories.append(os.getcwd()+'/ExperimentationRSAST/results_rsast/Server17_Hyperparameter_Tunning')


# Create an empty list to store the dataframes
dfs = []

# Loop through all files in the directory
for directory in directories:
    for filename in os.listdir(directory):
        # Check if the file starts with "df_overall_tunning" and ends with ".csv"
        if filename.startswith("df_all_overall_tunning") and filename.endswith(".csv"):
            # Read the csv file into a dataframe
            filepath = os.path.join(directory, filename)
            df = pd.read_csv(filepath)
            df['rpoint']=df['rpoint'].astype(str)
            df['nb_per_class']=df['nb_per_class'].astype(str)
            df['rpoint']=df['rpoint'].replace("(lenthg ts)//2","half_len")
            df['nb_per_class']=df['nb_per_class'].replace("(max instances per class)//2","half_instance")
            df['classifier_name']=df['classifier_name'].str.replace("\(lenthg ts\)//2","half_len")
            df['classifier_name']=df['classifier_name'].str.replace("\(max instances per class\)//2","half_instance")
            df['directory']=directory
            # Append the dataframe to the list
            dfs.append(df)

# Concatenate all the dataframes in the list into one dataframe
df_result = pd.concat(dfs, ignore_index=True)
# df_result.head(3)


In [14]:
hyperparameter='ACF&PACF: n_random_points=10 nb_inst_per_class=10'

In [15]:
df_result=df_result[['accuracy','dataset_name','classifier_name','time']]
df_result=df_result[df_result["classifier_name"]==hyperparameter]
df_result['classifier_name'].replace(hyperparameter,"RSAST",inplace=True)
hyperparameter="RSAST"
df_result.dataset_name.unique()

array(['Adiac', 'ArrowHead', 'Beef', 'BeetleFly', 'BirdChicken', 'BME',
       'Car', 'CBF', 'Chinatown', 'ChlorineConcentration', 'Coffee',
       'CricketX', 'CricketY', 'CricketZ', 'Crop', 'DiatomSizeReduction',
       'DistalPhalanxOutlineAgeGroup', 'DistalPhalanxOutlineCorrect',
       'DistalPhalanxTW', 'DodgerLoopDay', 'DodgerLoopGame',
       'DodgerLoopWeekend', 'Earthquakes', 'ECG200', 'ECG5000',
       'ECGFiveDays', 'ElectricDevices', 'FaceAll', 'FaceFour',
       'FacesUCR', 'FiftyWords', 'FreezerRegularTrain',
       'FreezerSmallTrain', 'Fungi', 'GunPoint', 'GunPointAgeSpan',
       'GunPointMaleVersusFemale', 'GunPointOldVersusYoung', 'Ham',
       'Herring', 'InsectWingbeatSound', 'ItalyPowerDemand', 'Mallat',
       'Meat', 'MedicalImages', 'MelbournePedestrian',
       'MiddlePhalanxOutlineAgeGroup', 'MiddlePhalanxOutlineCorrect',
       'MiddlePhalanxTW', 'MoteStrain', 'PhalangesOutlinesCorrect',
       'PLAID', 'Plane', 'PowerCons', 'ProximalPhalanxOutlineAgeGroup'

In [16]:
len(df_result.dataset_name.unique())

75

In [17]:
df_other_methods=df_other_methods.rename(columns={"dataset":"dataset_name","score":"accuracy","method":"classifier_name"})
df_other_methods.classifier_name.unique()

array(['BoP_UCR_10RES', 'BOSS_UCR_10RES', 'CID_DTW_UCR_10RES',
       'DDTW_UCR_10RES', 'DD_DTW_UCR_10RES', 'DTD_C_UCR_10RES',
       'DTW_F_UCR_10RES', 'EE_UCR_10RES', 'Flat-COTE_UCR_10RES',
       'FS_UCR_10RES', 'HIVE-COTE_UCR_10RES', 'LPS_UCR_10RES',
       'LS_UCR_10RES', 'MSM_UCR_10RES', 'RotF_UCR_10RES',
       'SAXVSM_UCR_10RES', 'ST_UCR_10RES', 'TSBF_UCR_10RES',
       'TSF_UCR_10RES', 'TWE_UCR_10RES', 'WDTW_UCR_10RES',
       'NB_UCR_TRAIN_TEST', 'C45_UCR_TRAIN_TEST', 'SVML_UCR_TRAIN_TEST',
       'SVMQ_UCR_TRAIN_TEST', 'BN_UCR_TRAIN_TEST', 'RandF_UCR_TRAIN_TEST',
       'RotF_UCR_TRAIN_TEST', 'MLP_UCR_TRAIN_TEST',
       'Euclidean_1NN_UCR_TRAIN_TEST', 'DTW_R1_1NN_UCR_TRAIN_TEST',
       'DTW_Rn_1NN_UCR_TRAIN_TEST', 'DDTW_R1_1NN_UCR_TRAIN_TEST',
       'DDTW_Rn_1NN_UCR_TRAIN_TEST', 'ERP_1NN_UCR_TRAIN_TEST',
       'LCSS_1NN_UCR_TRAIN_TEST', 'MSM_1NN_UCR_TRAIN_TEST',
       'TWE_1NN_UCR_TRAIN_TEST', 'WDDTW_1NN_UCR_TRAIN_TEST',
       'WDTW_1NN_UCR_TRAIN_TEST', 'DD_DTW_UCR_TRA

In [18]:

#Comparison RSAST, SAST and STC (No resampling)
df_other_methods=df_other_methods[df_other_methods['classifier_name'].isin(['ST_UCR_TRAIN_TEST','SAST_PAPERSAST_TRAIN_TEST','DTW_R1_1NN_UCR_TRAIN_TEST'])]
#Comparison RSAST and Shapelet Methods (10 resampling)
#df_other_methods=df_other_methods[df_other_methods['classifier_name'].isin(['RDST_RSAST_10RES','FS_UCR_10RES', 'LS_UCR_10RES','DTW_R1_1NN_UCR_TRAIN_TEST'])]
#Comparison RSAST and Other Categories (distance, interval, dictionary)(10 resampling)
#df_other_methods=df_other_methods[df_other_methods['classifier_name'].isin(['EE_UCR_10RES', 'TSBF_UCR_10RES','BOSS_UCR_10RES','DTW_R1_1NN_UCR_TRAIN_TEST'])]
#Comparison RSAST and SOTA methods(10 resampling)
#df_other_methods=df_other_methods[df_other_methods['classifier_name'].isin(['Rocket_RSAST_10RES','HCTE_PAPERROCKET_AVG_10RES','CHIEF_PAPERROCKET_AVG_10RES','DTW_R1_1NN_UCR_TRAIN_TEST'])]

#df_other_methods=df_other_methods[df_other_methods['classifier_name'].isin(['R_DST_PAPERRDST_AVG_30RES','RDST_RSAST_10RES','Rocket_RSAST_10RES','Rocket_PAPERROCKET_AVG_10RES','HIVE-COTE_UCR_10RES','HCTE_PAPERROCKET_AVG_10RES'])]
df_other_methods['classifier_name'].unique()

array(['DTW_R1_1NN_UCR_TRAIN_TEST', 'ST_UCR_TRAIN_TEST',
       'SAST_PAPERSAST_TRAIN_TEST'], dtype=object)

In [19]:

df_other_methods['classifier_name'].replace("DTW_R1_1NN_UCR_TRAIN_TEST","1NN-DTW",inplace=True)
df_other_methods['classifier_name'].replace("RotF_UCR_TRAIN_TEST","RotF",inplace=True)
df_other_methods['classifier_name'].replace("ST_UCR_TRAIN_TEST","STC",inplace=True)
df_other_methods['classifier_name'].replace("SAST_PAPERSAST_TRAIN_TEST","SAST",inplace=True)
df_other_methods['classifier_name'].replace("FS_UCR_TRAIN_TEST","FS",inplace=True)
df_other_methods['classifier_name'].replace("LS_UCR_TRAIN_TEST","LS",inplace=True)
df_other_methods['classifier_name'].replace("BOSS_UCR_TRAIN_TEST","BOSS",inplace=True)
df_other_methods['classifier_name'].replace("TSBF_UCR_TRAIN_TEST","TSBF",inplace=True)
df_other_methods['classifier_name'].replace("EE_UCR_TRAIN_TEST","EE",inplace=True)
df_other_methods['classifier_name'].replace("HIVE-COTE_UCR_TRAIN_TEST","HIVE-COTE",inplace=True)
df_other_methods['classifier_name'].replace("Rocket_RSAST_TRAIN_TEST","ROCKET",inplace=True)
df_other_methods['classifier_name'].replace("TSBF_UCR_TRAIN_TEST","TSBF",inplace=True)
df_other_methods['classifier_name'].replace("ResNet_PAPERROCKET_AVG_10RES","ResNet",inplace=True)
df_other_methods['classifier_name'].replace("HCTE_PAPERROCKET_AVG_10RES","HIVE-COTE",inplace=True)
df_other_methods['classifier_name'].replace("ITime_PAPERROCKET_AVG_10RES","InceptionTime",inplace=True)
df_other_methods['classifier_name'].replace("CHIEF_PAPERROCKET_AVG_10RES","TS-CHIEF",inplace=True)
df_other_methods['classifier_name'].replace("Rocket_PAPERROCKET_AVG_10RES","ROCKET",inplace=True)
df_other_methods['classifier_name'].replace("BOSS_PAPERROCKET_AVG_10RES","BOSS",inplace=True)
df_other_methods['classifier_name'].replace("ST_PAPERROCKET_AVG_10RES","ST",inplace=True)
df_other_methods['classifier_name'].replace("R_DST_PAPERRDST_AVG_30RES","RDST",inplace=True)
df_other_methods['classifier_name'].replace("FS_UCR_10RES","FS",inplace=True)
df_other_methods['classifier_name'].replace("LS_UCR_10RES","LS",inplace=True)
df_other_methods['classifier_name'].replace("BOSS_UCR_10RES","BOSS",inplace=True)
df_other_methods['classifier_name'].replace("EE_UCR_10RES","EE",inplace=True)
df_other_methods['classifier_name'].replace("TSBF_UCR_10RES","TSBF",inplace=True)
df_other_methods['classifier_name'].replace("RDST_RSAST_10RES","RDST",inplace=True)
df_other_methods['classifier_name'].replace("Rocket_RSAST_10RES","ROCKET",inplace=True)


In [20]:
df_other_methods['classifier_name'].unique()

array(['1NN-DTW', 'STC', 'SAST'], dtype=object)

In [21]:
# concatenate the two dataframes
concatenated_df = pd.concat([df_other_methods, df_result])

# reset the index of the concatenated dataframe
concatenated_df = concatenated_df.reset_index(drop=True)


In [22]:
concatenated_df['classifier_name'].unique()

array(['1NN-DTW', 'STC', 'SAST', 'RSAST'], dtype=object)

In [23]:
len(concatenated_df.dataset_name.unique())

103

### Overall analysis

#### Accuracy: RSAST vs Other methods

In [24]:
max_bx=max(df_result.accuracy)
min_bx=min(df_result.accuracy)



# Plot
fig, ax = plt.subplots()


#order=list(df_to_cd)
sns.boxplot(data=df_result, y='dataset_name', x='accuracy')
ax.set_xticklabels(ax.get_xticklabels(), rotation=90, ha='right')

# Axis details

ax.set(ylabel='dataset name', xlabel='accuracy', title='Accuracy Boxplot for Dataset:'+str(df_result["classifier_name"].unique()[0]))

# save plot
plt.savefig('ExperimentationRSAST/images_boxplot_acc/boxplot_acc_per_ds.png')

C:\Users\Surface pro\AppData\Local\Temp\ipykernel_16740\698087263.py:12: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels(ax.get_xticklabels(), rotation=90, ha='right')


In [25]:
filter_param=concatenated_df[concatenated_df.classifier_name==hyperparameter]

In [26]:
stats = filter_param.groupby(['dataset_name'])['accuracy'].agg(['mean', 'count', 'std'])

ci95_hi = []
ci95_lo = []
ci95_hi_lo = []

for i in stats.index:
    m, c, s = stats.loc[i]
    ci95_hi.append(m + 1.96*s/math.sqrt(c))
    ci95_lo.append(m - 1.96*s/math.sqrt(c))
    ci95_hi_lo.append(1.96*s/math.sqrt(c))

#stats['ci95_hi'] = ci95_hi
#stats['ci95_lo'] = ci95_lo
stats['ci95_interval'] = ci95_hi_lo
print(stats.round(2))

              mean  count   std  ci95_interval
dataset_name                                  
Adiac         0.58      5  0.01           0.01
ArrowHead     0.79      5  0.01           0.01
BME           0.83      5  0.01           0.00
Beef          0.68      5  0.02           0.02
BeetleFly     0.77      5  0.03           0.02
...            ...    ...   ...            ...
TwoPatterns   1.00      5  0.00           0.00
UMD           0.97      5  0.01           0.01
Wafer         1.00      5  0.00           0.00
Wine          0.78      5  0.04           0.03
WordSynonyms  0.68      5  0.00           0.00

[75 rows x 4 columns]


In [27]:
# create a pivot table with the mean of score by hyperparameter
t_score_by_method=pd.pivot_table(concatenated_df, values='accuracy', columns=['classifier_name'],index=['dataset_name'], aggfunc='mean')
t_score_by_method.iloc[0:25,:]


classifier_name,1NN-DTW,RSAST,SAST,STC
dataset_name,,,,
Adiac,0.603581,0.576982,0.68,0.782609
ArrowHead,0.702857,0.788571,0.77,0.737143
BME,NaN,0.832000,0.87,NaN
Beef,0.633333,0.680000,0.80,0.900000
BeetleFly,0.700000,0.770000,0.80,0.900000
BirdChicken,0.750000,0.850000,0.76,0.800000
CBF,0.996667,0.991333,0.98,0.974444
Car,0.733333,0.836667,0.88,0.916667
Chinatown,NaN,0.976093,0.96,NaN


In [28]:
t_score_by_method.iloc[25:49,:]

classifier_name,1NN-DTW,RSAST,SAST,STC
dataset_name,,,,
ECG5000,0.924444,0.937289,0.94,0.943778
ECGFiveDays,0.767712,0.995354,1.00,0.983740
Earthquakes,0.719424,0.717986,0.68,0.741007
ElectricDevices,0.602127,0.571807,0.62,0.746985
FaceAll,0.807692,0.783077,0.78,0.778698
FaceFour,0.829545,1.000000,1.00,0.852273
FacesUCR,0.904878,0.960293,0.95,0.905854
FiftyWords,0.690110,0.770110,0.77,0.705495
Fish,0.822857,NaN,NaN,0.988571


In [29]:
t_score_by_method.iloc[49:75,:]

classifier_name,1NN-DTW,RSAST,SAST,STC
dataset_name,,,,
ItalyPowerDemand,0.950437,0.942857,0.96,0.947522
LargeKitchenAppliances,0.794667,NaN,NaN,0.858667
Lightning2,0.868852,NaN,NaN,0.737705
Lightning7,0.726027,NaN,NaN,0.726027
Mallat,0.933902,0.913433,NaN,0.964179
Meat,0.933333,0.936667,0.92,0.850000
MedicalImages,0.736842,0.637632,0.68,0.669737
MelbournePedestrian,NaN,0.884789,0.87,NaN
MiddlePhalanxOutlineAgeGroup,0.500000,0.506494,0.53,0.642857


In [30]:
t_score_by_method.iloc[25:99,:]

classifier_name,1NN-DTW,RSAST,SAST,STC
dataset_name,,,,
ECG5000,0.924444,0.937289,0.94,0.943778
ECGFiveDays,0.767712,0.995354,1.00,0.983740
Earthquakes,0.719424,0.717986,0.68,0.741007
ElectricDevices,0.602127,0.571807,0.62,0.746985
FaceAll,0.807692,0.783077,0.78,0.778698
...,...,...,...,...
UWaveGestureLibraryX,0.727527,NaN,NaN,0.802903
UWaveGestureLibraryY,0.634003,NaN,NaN,0.730318
UWaveGestureLibraryZ,0.658291,NaN,NaN,0.748465


In [31]:
t_score_by_method.iloc[99:128,:]

classifier_name,1NN-DTW,RSAST,SAST,STC
dataset_name,,,,
WordSynonyms,0.648903,0.684953,0.7,0.570533
Worms,0.584416,NaN,NaN,0.740260
WormsTwoClass,0.623377,NaN,NaN,0.831169
Yoga,0.836667,NaN,NaN,0.817667


In [32]:
t_score_by_method.describe().round(2)

classifier_name,1NN-DTW,RSAST,SAST,STC
count,85.00,75.00,72.00,85.00
mean,0.74,0.84,0.84,0.82
std,0.17,0.13,0.13,0.14
min,0.23,0.51,0.53,0.32
25%,0.64,0.77,0.76,0.74
50%,0.75,0.87,0.87,0.84
75%,0.86,0.96,0.96,0.95
max,1.00,1.00,1.00,1.00


In [33]:
t_score_by_method=pd.pivot_table(concatenated_df, values='accuracy', index=['classifier_name'],columns=['dataset_name'], aggfunc='mean')
t_score_by_method=t_score_by_method.transpose()#[[hyperparameter,"SAST"]].round(2)

In [34]:
pd.pivot_table(concatenated_df, values='accuracy', index=['classifier_name'], aggfunc='var').round(4)

,accuracy
classifier_name,
1NN-DTW,0.0284
RSAST,0.0182
SAST,0.0164
STC,0.0210


In [35]:
pv_all=pd.pivot_table(df_result,index=['dataset_name'],values='accuracy',aggfunc="count")
ds_complete=pv_all[pv_all.accuracy==max(pv_all.accuracy)].reset_index().dataset_name.unique()
df_result=df_result[df_result.dataset_name.isin(ds_complete)]

df_other_methods=df_other_methods[df_other_methods.dataset_name.isin(ds_complete)]

pv_all=pd.pivot_table(df_other_methods,index=['dataset_name'],values='accuracy',aggfunc="count")
ds_complete=pv_all[pv_all.accuracy==max(pv_all.accuracy)].reset_index().dataset_name.unique()
df_other_methods=df_other_methods[df_other_methods.dataset_name.isin(ds_complete)]

df_result=df_result[df_result.dataset_name.isin(ds_complete)]
# concatenate the two dataframes
concatenated_df = pd.concat([df_other_methods, df_result])

# reset the index of the concatenated dataframe
concatenated_df = concatenated_df.reset_index(drop=True)

In [36]:
#concatenated_df=concatenated_df[concatenated_df.dataset_name!="Fungi"]
df_perf=pd.pivot_table(concatenated_df, values='accuracy', index=['classifier_name','dataset_name'], aggfunc='mean')

In [37]:


df_perf=df_perf.reset_index()
pd.pivot_table(df_perf, values='accuracy', index=['classifier_name'], aggfunc='count')

,accuracy
classifier_name,
1NN-DTW,56
RSAST,56
SAST,56
STC,56


In [38]:
#concatenated_df['classifier_name']=concatenated_df['classifier_name'].str.split('_').str[0]
score_by_method=pd.pivot_table(df_perf, values='accuracy', columns=['classifier_name'],index=['dataset_name'], aggfunc='mean')
score_by_method

classifier_name,1NN-DTW,RSAST,SAST,STC
dataset_name,,,,
Adiac,0.603581,0.576982,0.68,0.782609
ArrowHead,0.702857,0.788571,0.77,0.737143
Beef,0.633333,0.680000,0.80,0.900000
BeetleFly,0.700000,0.770000,0.80,0.900000
BirdChicken,0.750000,0.850000,0.76,0.800000
CBF,0.996667,0.991333,0.98,0.974444
Car,0.733333,0.836667,0.88,0.916667
ChlorineConcentration,0.648438,0.769531,0.75,0.699740
Coffee,1.000000,1.000000,1.00,0.964286


In [39]:
#hyperparameter='RDST_30RES'

In [40]:
t_score_by_method=pd.pivot_table(df_perf, values='accuracy', index=['classifier_name'],columns=['dataset_name'], aggfunc='mean')
t_score_by_method=t_score_by_method.transpose()#[[hyperparameter,"SAST"]].round(2)
for methods in t_score_by_method.columns.drop(hyperparameter):
    print("method: "+methods+" vs "+hyperparameter)
    total_wins=len(t_score_by_method[0<(t_score_by_method[hyperparameter].round(2)-t_score_by_method[methods].round(2))])
    print("winning "+hyperparameter+": "+str(total_wins))
    total_loss=len(t_score_by_method[0>(t_score_by_method[hyperparameter].round(2)-t_score_by_method[methods].round(2))])
    print("losse "+hyperparameter+": "+str(total_loss))
    total_draws=len(t_score_by_method[0==(t_score_by_method[hyperparameter].round(2)-t_score_by_method[methods].round(2))])
    print("draws: "+str(total_draws))

method: 1NN-DTW vs RSAST
winning RSAST: 41
losse RSAST: 10
draws: 5
method: SAST vs RSAST
winning RSAST: 24
losse RSAST: 21
draws: 11
method: STC vs RSAST
winning RSAST: 20
losse RSAST: 30
draws: 6


In [41]:
delete_files_in_directory("ExperimentationRSAST/images_one_vs_one_comparison/")
for method in score_by_method.columns.drop(hyperparameter):
    y=score_by_method[hyperparameter]
    x=score_by_method[method]

    # set the style of the plot
    plt.style.use('seaborn')

    # create the plot
    fig, ax = plt.subplots(figsize=(8, 6))

    # plot the line
    ax.plot([0,1], [0,1])

    # plot the points
    ax.scatter(x, y, color='blue', edgecolor='blue', s=30, zorder=10)

    # add labels and title
    ax.set_ylabel(hyperparameter)
    ax.set_xlabel(method)
    ax.set_title('Accuracy')


    # set the background color
    ax.set_facecolor('lightgrey')
    plt.savefig('ExperimentationRSAST/images_one_vs_one_comparison/'+hyperparameter+'vs'+method+'.png')


All files deleted successfully.


C:\Users\Surface pro\AppData\Local\Temp\ipykernel_16740\3386241549.py:7: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')
C:\Users\Surface pro\AppData\Local\Temp\ipykernel_16740\3386241549.py:7: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')
C:\Users\Surface pro\AppData\Local\Temp\ipykernel_16740\3386241549.py:7: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they w

In [42]:
# create a pivot table with the mean of time by hyperparameter
time_per_method=pd.pivot_table(df_result, values='time', columns=['classifier_name'],index=['dataset_name'], aggfunc='mean')/60
time_per_method

classifier_name,RSAST
dataset_name,
Adiac,37.744124
ArrowHead,0.754680
Beef,4.014065
BeetleFly,2.333761
BirdChicken,2.664224
CBF,0.128003
Car,9.757315
ChlorineConcentration,2.310386
Coffee,0.588011


#### Generate CD Diagram comparison

In [43]:
concatenated_df.classifier_name.unique()

array(['1NN-DTW', 'STC', 'SAST', 'RSAST'], dtype=object)

In [44]:


df_perf=pd.pivot_table(concatenated_df, values='accuracy', index=['classifier_name','dataset_name'], aggfunc='mean')
df_perf=df_perf.reset_index()
draw_cd_diagram(df_perf, labels=True, fname='ExperimentationRSAST/images_cd_diagram/cd-diagram_overall_comparison_other_method.png')


{0: '1NN-DTW', 1: 'RSAST', 2: 'SAST', 3: 'STC'}
[1, 2, 3]


In [45]:
df_perf["classifier_name"].unique()

array(['1NN-DTW', 'RSAST', 'SAST', 'STC'], dtype=object)

In [46]:
_, average_ranks, _ = wilcoxon_holm(df_perf=df_perf)
min_rank= min(average_ranks)
average_ranks=pd.DataFrame(average_ranks)
average_ranks

,0
1NN-DTW,3.339286
RSAST,2.339286
SAST,2.196429
STC,2.125000


In [47]:
uca_ds=pd.read_excel("ExperimentationRSAST/DataSetsUCLASummary.xlsx")

uca_ds.columns

Index(['USED SAST', 'BAKE OFF', 'N RUNS S17_S16_HT', 'N RUNS S17_SAST_DS',
       'N RUNS S17_SAST_DS_all', 'N RUNS S17_SAST_DS_extra_1',
       'N RUNS S17_SAST_DS_extra_2', 'N RUNS S17_OTHER_DEFAULT_SPLIT',
       'N RUNS S17_RSAST_R10', 'N RUNS S17_OTHER_R10', 'ID', 'Name', 'Type',
       'Train ', 'Test ', 'Class', 'Length', 'RSAST (10,10) WORST SAST?',
       'O(|c|nm²)', 'O(|c|nm³)', 'Approx Time (hours)', 'L25', 'L50', 'L75',
       'ED (w=0)', 'DTW (learned_w) ', 'DTW (w=100)', 'Default rate',
       'Data donor/editor', '50/L', 'Unnamed: 30'],
      dtype='object')

In [48]:
concatenated_df=concatenated_df.merge(uca_ds, left_on=['dataset_name'],right_on=['Name'])
concatenated_df.columns

Index(['dataset_name', 'accuracy', 'classifier_name', 'time', 'USED SAST',
       'BAKE OFF', 'N RUNS S17_S16_HT', 'N RUNS S17_SAST_DS',
       'N RUNS S17_SAST_DS_all', 'N RUNS S17_SAST_DS_extra_1',
       'N RUNS S17_SAST_DS_extra_2', 'N RUNS S17_OTHER_DEFAULT_SPLIT',
       'N RUNS S17_RSAST_R10', 'N RUNS S17_OTHER_R10', 'ID', 'Name', 'Type',
       'Train ', 'Test ', 'Class', 'Length', 'RSAST (10,10) WORST SAST?',
       'O(|c|nm²)', 'O(|c|nm³)', 'Approx Time (hours)', 'L25', 'L50', 'L75',
       'ED (w=0)', 'DTW (learned_w) ', 'DTW (w=100)', 'Default rate',
       'Data donor/editor', '50/L', 'Unnamed: 30'],
      dtype='object')

In [49]:
acc_per_type=pd.pivot_table(concatenated_df, values="accuracy", index=["dataset_name","Type",'classifier_name'])
#acc_per_type=concatenated_df
acc_per_type=acc_per_type.reset_index()

In [50]:
max_bx=max(acc_per_type.accuracy)
min_bx=min(acc_per_type.accuracy)


#df_to_cd=pd.pivot_table(df_to_cd, index=['dataset_name','classifier_name'], columns=['rpoint'],values='accuracy')
#df_to_cd = df_to_cd.reindex(columns=["1","10","30","50","100","half_len"])

#print(df_to_cd.head(5))

# Plot
fig, ax = plt.subplots()


#order=list(df_to_cd)
sns.boxplot(data=acc_per_type, y='Type', x='accuracy', hue ='classifier_name')
#plt.boxplot(df_to_cd, labels=list(df_to_cd), showfliers=False)
#ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')

# Axis details

ax.set(ylabel='dataset name', xlabel='accuracy', title='Accuracy Boxplot for Dataset:'+str(df_result["classifier_name"].unique()[0]))

#plt.xticks([1, 2, 3, 4, 5, 6],list(df_to_cd) )
#print(df_to_cd.describe())
plt.tight_layout()
# save plot
plt.savefig('ExperimentationRSAST/images_boxplot_acc_per_type/boxplot_acc_per_type.png')

In [51]:
pd.pivot_table(acc_per_type,index='Type', aggfunc=lambda x: len(x.unique()))

,accuracy,classifier_name,dataset_name
Type,,,
Device,4,4,1
ECG,16,4,4
Image,83,4,24
Motion,21,4,6
Sensor,34,4,11
Simulated,15,4,4
Spectro,21,4,6
